In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 18 20:18:15 2020

@author: jeffr
"""

# packages needed
import os
import pandas as pd

from pytube import YouTube
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *


In [ ]:
# import dataframe of powerhour videos
df = pd.read_csv('./powerhour.csv')

print(df.head(5))

In [ ]:
# create iterables for video loops
itrlink = df.link
itrduration = df.duration

df = df[0:4]

print(df.iloc[0].link)

VIDEO_STAGE_PATH = './video_staging/'


In [ ]:
# initialize empty dataframe to put metadata in
video_length_logging = pd.DataFrame(columns=['video_title', 'video_start', 'video_finish'])

#  download youtube videos
for clip in range(0,len(df)):
        
    video_link = df.iloc[clip].link
    clipid = df.iloc[clip].clipid
    yt = YouTube(video_link)

    # grab only video (mp4) files
    mp4files = yt.streams.filter('mp4')
    #
    d_video = yt.streams.first()#(mp4files[-1].extension,mp4files[-1].resolution)
    
    # build some metadata for slicing videos later
    video_title = d_video.title
    video_start = video_link.split("&t=",1)[1]
    video_finish = float(video_start) + df.iloc[clip].duration
    video_length_logging = video_length_logging.append(
        {'video_title': clipid, 
         'video_start': video_start, 
         'video_finish': video_finish}
        , ignore_index=True)
    
    try: 
        #downloading the video 
        d_video.download(VIDEO_STAGE_PATH, filename=clipid) 
    except: 
        print("Some Error!") 
    print('Task Completed!')


In [ ]:
# clip videos to relevant time periods

for video_clip_name in os.listdir('video_staging/'):
    
    video_metadata = video_length_logging[video_length_logging['video_title']==video_clip_name.split(".mp4",1)[0]]
    
    test_clip = "video_staging/" + video_clip_name
    trimmed_clip = ffmpeg_extract_subclip(test_clip,
                                          float(video_metadata['video_start']),
                                          float(video_metadata['video_finish']),
                                          targetname='./video_clipping/' + video_clip_name)


In [ ]:
# Intialize empty set for videos
L =[]

for root, dirs, files in os.walk("video_clipping/"):

    files.sort(reverse=True)

    for file in files:
        if os.path.splitext(file)[1] == '.mp4':
            filePath = os.path.join(root, file)
            video = VideoFileClip(filePath)
            L.append(video)

final_clip = concatenate_videoclips(L)
final_clip.to_videofile("outputs/output.mp4", fps=24, remove_temp=False)